### Load Libraries

In [1]:
import os, sys
project_home = "/Users/pitehrhurtado/Documents/OperationResearchs/clustering_and_discretization/src/"
if project_home not in sys.path:
    sys.path = [project_home] + sys.path

In [2]:
import numpy as np
import pandas as pd

from app.aggregations.nearest_pixel import Meta_Pixel

import logging

##### Configs

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

### Load Data
#### A. pixel data

#### B. grid data

### Description of Layers

#### A. Current State

## Aggregation Process